In [ ]:
import csv
import os
import cobra
from cobra.util.solver import linear_reaction_coefficients
from cobra.io import read_sbml_model, write_sbml_model,load_matlab_model
from cobra.flux_analysis import fastcc
import itertools as it
import numpy as np
import pandas as pd


In [ ]:
os.environ['GUROBI_HOME'] = "/home/jiang817/cybergut_project/Software/gurobi_license"
os.environ['GRB_LICENSE_FILE'] = "/home/jiang817/cybergut_project/Software/gurobi_license/gurobi.lic"

In [ ]:
# Load 4 draft reconstructions obtained by iMAT, CORDA, tINIT and pymCADRE
draft_recon_iMAT = read_sbml_model('/home/jiang817/cybergut_project/Gapfilling/fastcc_outputs/reconstruction_iMAT_fastcc.xml')
draft_recon_CORDA = read_sbml_model('/home/jiang817/cybergut_project/Gapfilling/fastcc_outputs/reconstruction_CORDA_fastcc.xml')
draft_recon_tINIT = read_sbml_model('/home/jiang817/cybergut_project/Gapfilling/fastcc_outputs/reconstruction_tINIT_fastcc.xml')
draft_recon_pymcadre = read_sbml_model('/home/jiang817/cybergut_project/Gapfilling/fastcc_outputs/reconstruction_pymcadre_fastcc.xml')

In [ ]:
# Load the Recon3D model
General_model = read_sbml_model('/home/jiang817/cybergut_project/General_models/Recon3D.xml')

In [ ]:
draft_recon_iMAT.solver = 'gurobi'
draft_recon_CORDA.solver = 'gurobi'
draft_recon_tINIT.solver = 'gurobi'
draft_recon_pymcadre.solver = 'gurobi'

In [ ]:
# the input of metabolic tasks
met_task_df = pd.read_csv('/home/jiang817/cybergut_project/metabolic_task/input_metabolic_tasks.csv', header=0)
# met_task_df
rxn_id = met_task_df.iloc[:,0].to_list()
consumed_met = met_task_df.iloc[:,1].to_list()
produced_met = met_task_df.iloc[:,2].to_list()


In [ ]:
# componenet of Ham's medium
medium_df= pd.read_csv('/home/jiang817/cybergut_project/metabolic_task/HAM_medium_composition.csv', header=0)
medium_df.dropna(axis=0, inplace=True)
HAM = medium_df.iloc[:,1].to_list()
HAM

In [ ]:
def test_metabolic_task(model, consumed_met, produced_met, General_model, Medium):

# model: draft reconstruction
# tasks: dict['consumed_metabolites']: 'produced metabolites'
# General_model: models that draft reconstructions came from
# Medium: boolen--True: using HAM medium; False: not using HAM medium  

    # close exchange reactions
    for i in range (len(model.boundary)):
        if model.boundary[i].id.startswith('EX_'):
            model.boundary[i].bounds = (0, 100)

    #  close all sinck reactions
    for i in range (len(model.boundary)):
        if model.boundary[i].id.startswith('SK_'):
            model.boundary[i].bounds = (0, 0)

   # close demand reactions
    for i in range (len(model.boundary)):
        if model.boundary[i].id.startswith('DM_'):
            model.boundary[i].lower_bound = 0

    # aerobic
    if 'EX_o2_e' not in [model.reactions[i].id for i in range(len(model.reactions))]:
        model.add_reactions([General_model.reactions.get_by_id('EX_o2_e')])
    model.reactions.get_by_id('EX_o2_e').bounds = (-40, 0)
    # change all reaction coefficient to 0
    Cofficient = {}
    for i in model.reactions:
        Cofficient[i] = 0
    model.objective = Cofficient

    # using Ham medium
    if Medium:
        model_boundary = [model.boundary[i].id for i in range(len(model.boundary))]
        for c in HAM:
            if c in model_boundary:
                model.reactions.get_by_id(c).lower_bound = -1
            else:
                c_met = c.split('EX_')[1]
                print(c)
                print(c_met + ": Ham component not in draft reconstrcution")
                model.add_boundary(General_model.metabolites.get_by_id(c_met), type="exchange")
                model.reactions.get_by_id(c).bounds = (-1, 100)
    
    
    tol = 1e-6
    
    # outputs
    TestSolution = np.array([])
    TestSolutionNam = np.array([])
    TestedRxns = np.array([])
   
    for i in range(len(consumed_met)):
        modelOri_copy = model.copy()
        # removed objective 
        Cofficient = {}
        for n in modelOri_copy.reactions:
            Cofficient[n] = 0
        modelOri_copy.objective = Cofficient
      
        met_cons = consumed_met[i]
        print(met_cons)
        if met_cons not in list(modelOri_copy.metabolites[i].id for i in range(len(modelOri_copy.metabolites))):
            print('not included')
            modelOri_copy.add_metabolites(General_model.metabolites.get_by_id(met_cons)) 
        
        met_prod = produced_met[i]
        print(met_prod)
        if met_prod not in list(modelOri_copy.metabolites[i].id for i in range(len(modelOri_copy.metabolites))):
            print('not included')
            modelOri_copy.add_metabolites(General_model.metabolites.get_by_id(met_prod)) 

        task_rxn_consumed = "SK_" + met_cons
        task_rxn_produced = "SK_" + met_prod   
        # add the above metabolties to sink reactions
        # set the boundary of consumed metabolite sink rxn with (-1, -1)
        # set the boundary of produced metabolite sink rxn with (0, 100)
        if task_rxn_consumed not in list(modelOri_copy.boundary[i].id for i in range(len(modelOri_copy.boundary))): 
            modelOri_copy.add_boundary(modelOri_copy.metabolites.get_by_id(met_cons), type="sink")
        if task_rxn_produced not in list(modelOri_copy.boundary[i].id for i in range(len(modelOri_copy.boundary))): 
            modelOri_copy.add_boundary(modelOri_copy.metabolites.get_by_id(met_prod), type="sink")
        modelOri_copy.reactions.get_by_id(task_rxn_consumed).bounds = (-1, -1)
        modelOri_copy.reactions.get_by_id(task_rxn_produced).bounds = (0, 100)

        # set the produced metabolite sink rxn as objective and then optimize the model
        modelOri_copy.objective = task_rxn_produced 
        print(modelOri_copy.objective.expression)
        print(modelOri_copy.objective.direction)
        FBA = modelOri_copy.optimize('maximize')
        TestSolution = np.append(TestSolution, FBA.objective_value)
        formula = rxn_id[i] + ': ' + met_cons + ' -> ' + met_prod
        TestSolutionNam = np.append(TestSolutionNam, formula)
        
        print(TestSolution)
        if TestSolution[-1] is not None:
            for m in range(len(FBA.fluxes)):
                if FBA.fluxes[m] > tol:
                    TestedRxns = np.append(TestedRxns, modelOri_copy.reactions[m].id)
    
    solution_final = pd.DataFrame(TestSolution, TestSolutionNam)
    TestedRxns  = np.unique(TestedRxns)
    model_rxn = np.array([model.reactions[r].id for r in range(len(model.reactions))])
    TestedRxns = np.intersect1d(TestedRxns,model_rxn)  
    PercTestedRxns = len(TestedRxns)*100/len(model_rxn)
    
    return solution_final, PercTestedRxns


In [ ]:
# solution, Percent = test_metabolic_task(draft_recon_iMAT, consumed_met, produced_met, General_model, True)
# solution, Percent = test_metabolic_task(draft_recon_CORDA, consumed_met, produced_met, General_model, True)
# solution, Percent = test_metabolic_task(draft_recon_tINIT, consumed_met, produced_met, General_model, True)
solution, Percent = test_metabolic_task(draft_recon_pymcadre, consumed_met, produced_met, General_model, True)

In [ ]:
solution